In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error  as MSE
from sklearn.metrics import mean_absolute_error as MAE
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
import sys

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#Question 1
train_data_name=input("Enter the name of train data file [(ex) harris.dat] : ") # data name
test_data_name=input("Enter the name of test data file [(ex) harris.dat] : ") 
coding_fm=int(input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")) # data separator
separator_fm={coding_fm ==1 : " "}.get(True, ",")
res_pos=int(input("Enter the column position of the response variable : [from 1 to p] : "))
header=input("Does the data have column header? (y/n) : ")
if(header=="y") : trdata=pd.read_csv(train_data_name, sep=separator_fm) # loading data
else : trdata=pd.read_csv(train_data_name, sep=separator_fm, header=None) # loading data
if(header=="y") : tstdata=pd.read_csv(test_data_name, sep=separator_fm) # loading data
else : tstdata=pd.read_csv(test_data_name, sep=separator_fm, header=None) # loading data
out_name=input("Enter the output file name to export [(ex) result.txt] : ")

# Extracting X and y
data = pd.DataFrame(trdata)
C = pd.DataFrame(np.ones(shape = (data.shape[0],)))
data = pd.concat([C,data], axis = 1, ignore_index= True)
X= data.drop([data.columns[res_pos]], axis=1 )
Y= data.iloc[:, res_pos ].values

#Extracting test X and y
tstdata = pd.DataFrame(tstdata)
tstC = pd.DataFrame(np.ones(shape = (tstdata.shape[0],)))
tstdata = pd.concat([tstC,tstdata], axis = 1, ignore_index= True)
tstX= tstdata.drop([tstdata.columns[res_pos]], axis=1 )
tstY= tstdata.iloc[:, res_pos ].values

Enter the name of train data file [(ex) harris.dat] : veh.dat
Enter the name of test data file [(ex) harris.dat] : vehtest.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 2
Enter the column position of the response variable : [from 1 to p] : 19
Does the data have column header? (y/n) : n
Enter the output file name to export [(ex) result.txt] : result.txt


In [194]:
def OVRfit(X,y):
  ovr = {}
  y1 = pd.DataFrame(index=range(0,len(y)), columns=range(0,len(np.unique(y))))
  for i in range(0, len(np.unique(y))):
    for j in range(0,len(y)):
        if y[j] == np.unique(y)[i]:
            y1.iloc[j, i] = 1
        else: 
            y1.iloc[j, i] = 0
  for i in range(0,len(np.unique(y))):
    ovr[f'logit{i}']= sm.Logit(y1[i].astype(float),X.astype(float) ).fit()

  return ovr


In [195]:
OVRfit(X,Y)

Optimization terminated successfully.
         Current function value: 0.360581
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.327391
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.042271
         Iterations 13
Optimization terminated successfully.
         Current function value: 0.027253
         Iterations 16


{'logit0': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7fa6e1741610>,
 'logit1': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7fa6e1717b50>,
 'logit2': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7fa6e1717d10>,
 'logit3': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7fa6e16af090>}

In [196]:
#2
def OVRpredict(tstX, tsty):
  predy = pd.DataFrame(index=range(0,len(tsty)), columns=range(0,len(np.unique(y))))
  for i in range(0, len(np.unique(y))):
    predy[i] = ovr[f'logit{i}'].predict(tstX)
  
  prediction = pd.DataFrame(index = range(0,len(tsty)), columns = range(0,1))
  for  i in range(0,len(tsty)):
    prediction[i] = np.argmax(predy.iloc[i])+1 
  prediction = prediction.iloc[0]

  y2 = pd.DataFrame(index=range(0,len(tsty)), columns=range(0,len(np.unique(y))))
  for i in range(0,len(tsty)):
    if np.sum(np.round(predy.iloc[i]/ np.sum(predy.iloc[i]),decimals = 1)) == 1:
      y2.iloc[i] = np.round(predy.iloc[i]/ np.sum(predy.iloc[i]),decimals = 1)  
    else:
      y2.iloc[i] = np.round(predy.iloc[i]/ np.sum(predy.iloc[i]),decimals = 1)
      y2.iloc[i][np.where(predy.iloc[i]==sorted(predy.iloc[i], reverse=True)[1])[0]]= 0.1

  return y2, prediction

In [197]:
A = OVRpredict(tstX,tstY)

In [198]:
#3
sys.stdout = open(out_name,'w')

k = 'ID, Actual Class'
split = k.split()
for i in range(0,len(np.unique(Y))):
  split.insert(i+2, f', Class {i+1}')
split.insert(-1,',Final prediction')
k = ' '.join(split)

print(k)
print('-----------------------------------')
for i in range(3):
  print(i+1, tstY[i],list(A[0].iloc[i]), A[1][i] )

print('skip: 중간은 생략')
for i in range(len(tstY)-3,len(tstY)):
  print(i+1, tstY[i],list(A[0].iloc[i]), A[1][i] )

In [199]:
#4
confusion_tst = confusion_matrix(tstY, A[1])

accu_tst = 0
for i in range(len(np.unique(Y))):
  accu_tst = accu_tst + confusion_tst[i][i]
accuracy_tst = accu_tst / tstX.shape[0]

print('\n\nconfusion matrix (test)')
print('----------------------------------')

print('           predicted class \n Actual 1 ' ,confusion_tst[0], '\n class  2 ', confusion_tst[1])
for i in range(2,len(np.unique(Y)) ):
  print(f'        {i+1} ', confusion_tst[i])
print('model summary')
print('----------------------------------')
print('Overall accuracy = ' ,accuracy_tst)